<a href="https://colab.research.google.com/github/kilos11/Learning_Transformers-/blob/main/Building_a_Tokenizer%2C_block_by_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##As we've seen in the previous sections, tokenization comprises several steps:

##*Normalization (any cleanup of the text that is deemed necessary, such as removing spaces or accents, Unicode normalization, etc.)*
##*Pre-tokenization (splitting the input into words)*
##*Running the input through the model (using the pre-tokenized words to produce a sequence of tokens)*
##*Post-processing (adding the special tokens of the tokenizer, generating the attention mask and token type IDs)*
##As a reminder, here's another look at the overall process:

##The 🤗 Tokenizers library has been built to provide several options for each of those steps, which you can mix and match together. In this section we'll see how we can build a tokenizer from scratch, as opposed to training a new tokenizer from an old one as we did in section 2. You'll then be able to build any kind of tokenizer you can think of!

##More precisely, the library is built around a central `Tokenizer` class with the building blocks regrouped in submodules:

##*`normalizers` contains all the possible types of `Normalizer` you can use (complete list here[link text](https://huggingface.co/docs/tokenizers/api/normalizers)).*
##*`pre_tokenizers` contains all the possible types of `PreTokenizer` you can use (complete list here[link text](https://huggingface.co/docs/tokenizers/api/pre-tokenizers)).*
##*`models` contains the various types of Model you can use, like `BPE, WordPiece, and Unigram`(complete list here[link text](https://huggingface.co/docs/tokenizers/api/models)).*
##*`trainers` contains all the different types of `Trainer` you can use to train your model on a corpus (one per type of model; complete list here[link text](https://huggingface.co/docs/tokenizers/api/trainers)).*
##*`post_processors` contains the various types of `PostProcessor` you can use (complete list here[link text](https://huggingface.co/docs/tokenizers/api/post-processors)).*
##*`decoders` contains the various types of `Decoder` you can use to decode the outputs of tokenization (complete list here[link text](https://huggingface.co/docs/tokenizers/components#decoders)).*
##You can find the whole list of building blocks here.

#**Acquiring a corpus**#
##To train our new tokenizer, we will use a small corpus of text (so the examples run fast). The steps for acquiring the corpus are similar to the ones we took at the beginning of this chapter, but this time we'll use the WikiText-2 [link text](https://huggingface.co/datasets/wikitext)dataset:

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext",
                       name="wikitext-2-raw-v1",
                       split="train")

def get_training_corpus():
    # Iterate through the dataset in chunks of 1000
    for i in range(0, len(dataset), 1000):
        # Yield a slice of 1000 text entries from the dataset
        yield dataset[i : i + 1000]["text"]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

##The function `get_training_corpus()` is a generator that will yield batches of 1,000 texts, which we will use to train the tokenizer.

##🤗 Tokenizers can also be trained on text files directly. Here's how we can generate a text file containing all the texts/inputs from WikiText-2 that we can use locally:

In [ ]:
# Open a file named "wikitext-2.txt" in write mode with UTF-8 encoding
with open("wikitext-2.txt", "w", encoding="utf-8") as f:
    # Iterate through each item in the dataset
    for i in range(len(dataset)):
        # Write the "text" field of each dataset item to the file
        # Add a newline character after each text entry
        f.write(dataset[i]["text"] + "\n")

#**Building a WordPiece tokenizer from scratch**#
##To build a tokenizer with the 🤗 Tokenizers library, we start by instantiating a `Tokenizer` object with a `model`, then set its `normalizer, pre_tokenizer, post_processor, and decoder` attributes to the values we want.

##For this example, we'll create a Tokenizer with a WordPiece model:

In [ ]:
# Import necessary components from the tokenizers library
from tokenizers import (
    # Used for decoding tokenized sequences back to text
    decoders,
    # Provides various tokenization models
    models,
    # Used for text normalization (e.g., lowercase, unicode normalization)
    normalizers,
    # Handles initial tokenization steps (e.g., splitting on whitespace)
    pre_tokenizers,
    # Processes tokens before and after encoding/decoding
    processors,
    # Used for training tokenizer models
    trainers,
    # The main Tokenizer class
    Tokenizer,
)

# Create a new tokenizer using the WordPiece model
# WordPiece is a subword tokenization algorithm
# [UNK] is set as the token for unknown words
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

##We have to specify the `unk_token` so the model knows what to return when it encounters characters it hasn't seen before. Other arguments we can set here include the `vocab` of our model (we're going to train the model, so we don't need to set this) and `max_input_chars_per_word`, which specifies a maximum length for each word (words longer than the value passed will be split).

##The first step of tokenization is normalization, so let's begin with that. Since BERT is widely used, there is a `BertNormalizer` with the classic options we can set for BERT: `lowercase and strip_accents`, which are self-explanatory; `clean_text` to remove all control characters and replace repeating spaces with a single one; and `handle_chinese_chars`, which places spaces around Chinese characters. To replicate the `bert-base-uncased` tokenizer, we can just set this normalizer:

In [ ]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

##Generally speaking, however, when building a new tokenizer you won't have access to such a handy normalizer already implemented in the 🤗 Tokenizers library -- so let's see how to create the BERT normalizer by hand. The library provides a Lowercase normalizer and a `StripAccents` normalizer, and you can compose several normalizers using a `Sequence`:

In [ ]:
tokenizer.normalizer = normalizers.Sequence(
        [normalizers.NFD(),
         normalizers.Lowercase(),
         normalizers.StripAccents()]
        )


##We're also using an `NFD` Unicode normalizer, as otherwise the `StripAccents` normalizer won't properly recognize the accented characters and thus won't strip them out.

##As we've seen before, we can use the `normalize_str()` method of the `normalizer` to check out the effects it has on a given text:

In [ ]:
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


##To go further If you test the two versions of the previous normalizers on a string containing the unicode character `u"\u0085"` you will surely notice that these two normalizers are not exactly equivalent. To not over-complicate the version with `normalizers.Sequence` too much , we haven't included the Regex replacements that the `BertNormalizer` requires when the clean_text argument is set to True - which is the default behavior. But don't worry: it is possible to get exactly the same normalization without using the handy BertNormalizer by adding two normalizers.Replace's to the normalizers sequence.

##Next is the pre-tokenization step. Again, there is a prebuilt BertPreTokenizer that we can use:

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

##Or we can build it from scratch:

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

##Note that the Whitespace pre-tokenizer splits on whitespace and all characters that are not letters, digits, or the underscore character, so it technically splits on whitespace and punctuation:

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

##If you only want to split on whitespace, you should use the `WhitespaceSplit pre-tokenizer` instead:

In [ ]:
pre_tokenizer = pre_tokenizers.WhitespaceSplit()
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[("Let's", (0, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre-tokenizer.', (14, 28))]

##Like with normalizers, you can use a `Sequence` to compose several pre-tokenizers:

In [ ]:
pre_tokenizer = pre_tokenizers.Sequence(
        [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
        )
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")


[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

##The next step in the tokenization pipeline is running the inputs through the model. We already specified our model in the initialization, but we still need to train it, which will require a `WordPieceTrainer`. The main thing to remember when instantiating a trainer in 🤗 Tokenizers is that you need to pass it all the special tokens you intend to use -- otherwise it won't add them to the vocabulary, since they are not in the training corpus:

In [ ]:
# Define a list of special tokens used in many NLP tasks
special_tokens = [
    #Represents unknown words (words not in the vocabulary)
    "[UNK]",
    #Used for padding sequences to a fixed length
    "[PAD]",
    #Typically used at the start of a sequence (e.g., for classification tasks)
    "[CLS]",
    #Used to separate different parts of a sequence
    "[SEP]",
    #Used in masked language modeling tasks
    "[MASK]",
]

# Create a WordPieceTrainer object to train the tokenizer
# Set the vocabulary size to 25000 words/subwords
# Include the special tokens defined above
trainer = trainers.WordPieceTrainer(vocab_size=25000,
                                    special_tokens=special_tokens)


##As well as specifying the `vocab_size` and `special_tokens`, we can set the `min_frequency` (the number of times a token must appear to be included in the vocabulary) or change the `continuing_subword_prefix` (if we want to use something different from `##`).

##To train our model using the iterator we defined earlier, we just have to execute this command:

In [ ]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

##We can also use text files to train our tokenizer, which would look like this (we reinitialize the model with an empty `WordPiece` beforehand):

In [ ]:
tokenizer.model = models.WordPiece(unk_token="[UNK]")
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

##In both cases, we can then test the tokenizer on a text by calling the `encode()` method:

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']


##The `encoding` obtained is an `Encoding`, which contains all the necessary outputs of the tokenizer in its various attributes: `ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, and overflowing`.

##The last step in the tokenization pipeline is post-processing. We need to add the `[CLS]` token at the beginning and the `[SEP]` token at the end (or after each sentence, if we have a pair of sentences). We will use a `TemplateProcessor` for this, but first we need to know the IDs of the `[CLS] and [SEP]` tokens in the vocabulary:

In [ ]:
# Get the ID of the [CLS] token from the tokenizer
cls_token_id = tokenizer.token_to_id("[CLS]")

# Get the ID of the [SEP] token from the tokenizer
sep_token_id = tokenizer.token_to_id("[SEP]")

# Print the IDs of the [CLS] and [SEP] tokens
print(cls_token_id, sep_token_id)

2 3


##To write the template for the `TemplateProcessor`, we have to specify how to treat a single sentence and a pair of sentences. For both, we write the special tokens we want to use; the `first (or single) sentence is represented by $A`, while the `second sentence (if encoding a pair) is represented by $B`. For each of these (special tokens and sentences), we also specify the corresponding token type ID after a colon.

##The classic BERT template is thus defined as follows:

In [ ]:
# Set up the post-processor for the tokenizer
tokenizer.post_processor = processors.TemplateProcessing(
    # Template for single sequence: [CLS] at start, then the sequence, then [SEP]
    single=f"[CLS]:0 $A:0 [SEP]:0",

    # Template for pair of sequences: [CLS], first sequence, [SEP], second sequence, [SEP]
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",

    # Define the special tokens and their IDs
        special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],

)

##Note that we need to pass along the IDs of the special tokens, so the tokenizer can properly convert them to their IDs.

##Once this is added, going back to our previous example will give:

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '[SEP]']


##And on a pair of sentences, we get the proper result:

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '...', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


##We've almost finished building this tokenizer from scratch -- the last step is to include a decoder:

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

##Let's test it on our previous encoding:

In [ ]:
tokenizer.decode(encoding.ids)

##Great! We can save our tokenizer in a single JSON file like this:

In [ ]:
tokenizer.save("tokenizer.json")

##We can then reload that file in a `Tokenizer` object with the `from_file()` method:

In [ ]:
new_tokenizer = Tokenizer.from_file("tokenizer.json")

#×To use this tokenizer in 🤗 Transformers, we have to wrap it in a `PreTrainedTokenizerFast`. We can either use the generic class or, if our tokenizer corresponds to an existing model, use that class (here,` BertTokenizerFast`). If you apply this lesson to build a brand new tokenizer, you will have to use the first option.

##To wrap the tokenizer in a PreTrainedTokenizerFast, we can either pass the tokenizer we built as a tokenizer_object or pass the tokenizer file we saved as tokenizer_file. The key thing to remember is that we have to manually set all the special tokens, since that class can't infer from the tokenizer object which token is the mask token, the [CLS] token, etc.:

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

##If you are using a specific tokenizer class (like BertTokenizerFast), you will only need to specify the special tokens that are different from the default ones (here, none):

In [ ]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

##You can then use this tokenizer like any other 🤗 Transformers tokenizer. You can save it with the `save_pretrained()` method, or upload it to the Hub with the `push_to_hub()1 method.

##Now that we've seen how to build a WordPiece tokenizer, let's do the same for a BPE tokenizer. We'll go a bit faster since you know all the steps, and only highlight the differences.

#**Building a BPE tokenizer from scratch**#
##Let's now build a GPT-2 tokenizer. Like for the BERT tokenizer, we start by initializing a Tokenizer with a BPE model:

In [ ]:
tokenizer = Tokenizer(models.BPE())

##Also like for BERT, we could initialize this model with a vocabulary if we had one (we would need to pass the vocab and merges in this case), but since we will train from scratch, we don't need to do that. We also don't need to specify an unk_token because GPT-2 uses byte-level BPE, which doesn't require it.

##GPT-2 does not use a normalizer, so we skip that step and go directly to the pre-tokenization:

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

##The option we added to ByteLevel here is to not add a space at the beginning of a sentence (which is the default otherwise). We can have a look at the pre-tokenization of an example text like before:

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!")

##Next is the model, which needs training. For GPT-2, the only special token is the end-of-text token:

In [ ]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

Like with the `WordPieceTrainer`, as well as the `vocab_size` and `special_tokens`, we can specify the `min_frequency` if we want to, or if we have an `end-of-word suffix (like </w>)`, we can set it with end_of_word_suffix.

##This tokenizer can also be trained on text files:

In [ ]:
tokenizer.model = models.BPE()
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

##Let's have a look at the tokenization of a sample text:

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

##We apply the byte-level post-processing for the GPT-2 tokenizer as follows:

In [ ]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

##The `trim_offsets = False` option indicates to the post-processor that we should leave the offsets of tokens that begin with 'Ġ' as they are: this way the start of the offsets will point to the space before the word, not the first character of the word (since the space is technically part of the token). Let's have a look at the result with the text we just encoded, where `'Ġtest'` is the token at index 4:

In [ ]:
sentence = "Let's test this tokenizer."
encoding = tokenizer.encode(sentence)
start, end = encoding.offsets[4]
sentence[start:end]

##Finally, we add a byte-level decoder:

In [ ]:
tokenizer.decoder = decoders.ByteLevel()

##and we can double-check it works properly:

In [ ]:
tokenizer.decode(encoding.ids)

##Great! Now that we're done, we can save the tokenizer like before, and wrap it in a `PreTrainedTokenizerFast or GPT2TokenizerFast` if we want to use it in 🤗 Transformers:

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
            )

OR:

In [ ]:
from transformers import GPT2TokenizerFast

wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

#**Building a Unigram tokenizer from scratch**#
##Let's now build an XLNet tokenizer. Like for the previous tokenizers, we start by initializing a Tokenizer with a Unigram model:

In [ ]:
tokenizer = Tokenizer(models.Unigram())

##Again, we could initialize this model with a vocabulary if we had one.

##For the normalization, XLNet uses a few replacements (which come from SentencePiece):

In [ ]:
from tokenizers import Regex

tokenizer.normalizer = normalizers.Sequence(
    [# Replace double backticks with double quotes
     normalizers.Replace("``", '"'),
     # Replace double single quotes with double quotes
     normalizers.Replace("''", '"'),
     # Apply Unicode normalization (decomposition)
     normalizers.NFKD(),
     # Remove accents from characters
     normalizers.StripAccents(),
     # Replace multiple spaces with a single space
     normalizers.Replace(Regex(" {2,}"), " "),]
    )

##This replaces `` and '' with " and any sequence of two or more spaces with a single space, as well as removing the accents in the texts to tokenize.

##The pre-tokenizer to use for any SentencePiece tokenizer is `Metaspace`:

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

##We can have a look at the pre-tokenization of an example text like before:

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the pre-tokenizer!")

##Next is the model, which needs training. XLNet has quite a few special tokens:

In [ ]:
special_tokens = ["<cls>",
                  "<sep>",
                  "<unk>",
                  "<pad>",
                  "<mask>",
                  "<s>",
                  "</s>"]

trainer = trainers.UnigramTrainer(
    vocab_size=25000,
    special_tokens=special_tokens,
    unk_token="<unk>"
)

tokenizer.train_from_iterator(get_training_corpus(),
                              trainer=trainer)

##A very important argument not to forget for the `UnigramTrainer` is the `unk_token`. We can also pass along other arguments specific to the Unigram algorithm, such as the `shrinking_factor` for each step where we remove tokens (defaults to 0.75) or the `max_piece_length` to specify the maximum length of a given token (defaults to 16).

##This tokenizer can also be trained on text files:

In [ ]:
tokenizer.model = models.Unigram()
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

##Let's have a look at the tokenization of a sample text:

##A peculiarity of XLNet is that it puts the `<cls>` token at the end of the sentence, with a type ID of 2 (to distinguish it from the other tokens). It's padding on the left, as a result. We can deal with all the special tokens and token type IDs with a template, like for BERT, but first we have to get the IDs of the `<cls>` and `<sep>` tokens:

In [ ]:
cls_token_id = tokenizer.token_to_id("<cls>")
sep_token_id = tokenizer.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

##The template looks like this:

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
        single="$A:0 <sep>:0 <cls>:2",
        pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
        special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)],
                )


##And we can test it works by encoding a pair of sentences:

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences!")
print(encoding.tokens)
print(encoding.type_ids)

##Finally, we add a Metaspace decoder:

In [ ]:
tokenizer.decoder = decoders.Metaspace()

##and we're done with this tokenizer! We can save the tokenizer like before, and wrap it in a `PreTrainedTokenizerFast` or `XLNetTokenizerFast` if we want to use it in 🤗 Transformers. One thing to note when using `PreTrainedTokenizerFast` is that on top of the special tokens, we need to tell the 🤗 Transformers library to pad on the left:

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

##Or alternatively:

In [ ]:
from transformers import XLNetTokenizerFast

wrapped_tokenizer = XLNetTokenizerFast(tokenizer_object=tokenizer)